In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Download CSV") \
    .master("local[*]")\
    .getOrCreate()


In [2]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import *

In [3]:
Schema = StructType() \
	.add('gestor', 'string') \
    .add('canalOrigem', 'string') \
    .add('regiao', 'string') \
    .add('uf', 'string') \
    .add('cidade', 'string') \
    .add('sexo', 'string') \
    .add('faixaEtaria', 'string') \
    .add('anoAbertura', 'integer') \
    .add('mesAbertura', 'integer') \
    .add('dataAbertura', 'string') \
    .add('dataResposta', 'string') \
    .add('dataAnalise', 'string') \
    .add('dataRecusa', 'string') \
    .add('dataFinalizacao', 'date') \
    .add('prazoResposta', 'string') \
    .add('prazoAnaliseGestor', 'integer') \
    .add('tempoResposta', 'integer') \
    .add('nomeFantasia', 'string') \
    .add('segmentoMercado', 'string') \
    .add('area', 'string') \
    .add('assunto', 'string') \
    .add('grupoProblema', 'string') \
    .add('problema', 'string') \
    .add('comoContratou', 'string') \
    .add('procurouEmpresa', 'string') \
    .add('respondida', 'string') \
    .add('situacao', 'string') \
    .add('avaliacaoReclamacao', 'string') \
    .add('notaConsumidor', 'integer') \
    .add('analiseRecusa', 'string')

In [4]:
inputDF = spark \
          .readStream \
          .option("maxFilesPerTrigger", 1) \
          .option("header", True) \
          .option("sep", ";") \
          .schema(Schema) \
          .csv("C:\Projetos\data-master\csv") \
          .withColumn("datRefCarga", lit(substring(col('datafinalizacao'),1,7)))

In [5]:
query = inputDF.writeStream \
                .partitionBy('datRefCarga') \
                .format("parquet") \
                .option("path", "hdfs://namenode:9000/data/consumidor/teste") \
                .option("checkpointLocation", "hdfs://namenode:9000/tmp/checkpoint") \
                .start()

Py4JJavaError: An error occurred while calling o50.start.
: org.apache.hadoop.ipc.RemoteException(java.io.IOException): File /tmp/checkpoint/.metadata.ace29e3a-5c66-4c6b-9aaf-8958e6722ed5.tmp could only be written to 0 of the 1 minReplication nodes. There are 1 datanode(s) running and 1 node(s) are excluded in this operation.
	at org.apache.hadoop.hdfs.server.blockmanagement.BlockManager.chooseTarget4NewBlock(BlockManager.java:2219)
	at org.apache.hadoop.hdfs.server.namenode.FSDirWriteFileOp.chooseTargetForNewBlock(FSDirWriteFileOp.java:294)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getAdditionalBlock(FSNamesystem.java:2789)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.addBlock(NameNodeRpcServer.java:892)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.addBlock(ClientNamenodeProtocolServerSideTranslatorPB.java:574)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlocking

In [ ]:
query.awaitTermination()

In [2]:
df = spark.read.parquet('hdfs://namenode:9000/data/consumidor/silver/datRefCarga=2023-05')

In [5]:
df.repartition(1).write.mode('overwrite').option('header', True).csv('/home/besgam/consumidor/')